# Run gas grid simulation
## Define connection parameters for database

In [1]:
from dblayer import *

connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
)

## Retrieve co-simulation graph from the database

In [2]:
from dblayer.zerobnl.reader import *

reader = DBReader( connect )
sim = reader.read_from_db( 'GasGridSim' )

In [3]:
sim.nodes

,ToSet,ToGet,InitVal,Parameters,Wrapper,Dockerfile,Files,Local,Meta,Env
GasGrid,"[(gas_node282/p_kW, kW), (gas_node274/p_kW, kW...",[],"{'network_id': 3000, 'dbport': 5432, 'dbname':...",{},/c/Development/integrcity/DemoToolchain/3_simu...,/c/Development/integrcity/DemoToolchain/3_simu...,[],False,GasGridMeta,GasGridEnv
Consumers,[],"[(gas_node282, kW), (gas_node274, kW), (gas_no...",{'start_date': '2019-01-01 00:00:00+00:00'},{},/c/Development/integrcity/DemoToolchain/3_simu...,/c/Development/integrcity/DemoToolchain/3_simu...,[/c/Development/integrcity/DemoToolchain/3_sim...,False,ConsumersMeta,ConsumersEnv


## Make the 3DCityDB available in the simulation

In [4]:
sim.add_container_to_simulation( 'citydb-container' )

## Running the simulation

In [5]:
sim.run()

INFO :: Simulation finished in 53 min and 51 sec


## Retrieve the simulation results

In [6]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,X,GasGrid,gas_node195/P
1,X,GasGrid,gas_node36/P
2,OUT,Consumers,gas_node58
3,X,GasGrid,gas_node7/P
4,OUT,Consumers,gas_node159
...,...,...,...
682,OUT,Consumers,gas_node71
683,X,GasGrid,gas_pipe3-256/m_dot
684,X,GasGrid,gas_pipe38-233/m_dot
685,X,GasGrid,gas_node217/P


In [7]:
import logging
logging.getLogger("ZEROBNL").setLevel(logging.WARNING)

In [8]:
results = sim.get_results_by_pattern("X||GasGrid||*")
print( 'Found {} results.'.format( len( results ) ) )

Found 564 results.


# Write results to database

## Extract results for individual pipes

In [9]:
pipes_m_dot = {}

for name, ts in results.items():
    if 'm_dot' in name:
        sim_var_name = name.split('||')[2]
        pipe_name = sim_var_name.split('/')[0]
        pipes_m_dot[pipe_name] = list(ts.array)

## Retrieve information about pipes stored in database

In [10]:
db_access = DBAccess()
db_access.connect_to_citydb(connect)

In [11]:
RoundPipe = db_access.map_citydb_object_class( 'RoundPipe',
    table_name='utn9_ntw_feat_distrib_elem_pipe_round', schema='citydb_view' 
    )

NetworkToFeature = db_access.map_citydb_object_class( 'NetworkToFeature',
    table_name='utn9_network_to_network_feature', schema='citydb', user_defined = True
    )

pipes_db_data = db_access.join_citydb_objects(
    [ 'RoundPipe', 'NetworkToFeature' ], 
    conditions = [
        RoundPipe.id == NetworkToFeature.network_feature_id,
        NetworkToFeature.network_id == 3000        
        ]
    )

## Associate results to pipes stored in database

In [12]:
import datetime
from dblayer.helpers.scn.add_sim_results import *

sim_res_ts = []

for data in pipes_db_data:
    
    pipe = data[0]
    val_array_m_dot = pipes_m_dot[pipe.name]
    
    ts_m_dot = SimResultRegularTimeSeries(
        name = '{}_m_dot'.format(pipe.name),
        object_id = pipe.id,
        values_array = val_array_m_dot,
        values_unit = 'kg/s',
        temporal_extent_begin = datetime.datetime( 2020, 1, 1, 0, 0, 0 ),
        temporal_extent_end = datetime.datetime( 2020, 1, 1, 23, 0, 0 ),
        time_interval = 1,
        time_interval_unit = 'h',
        acquisition_method = 'Simulation',
        interpolation_type = 'AverageInSucceedingInterval',
        )

    sim_res_ts.append(ts_m_dot)

In [13]:
add_sim_results_regular_time_series(
    db_access,
    'GasGridSimResults',
    sim_res_ts
    )

1

In [14]:
db_access.commit_citydb_session()

In [15]:
del db_access